In [1]:
import json
import gzip
import os
from datetime import timedelta, date
import pandas as pd
from math import log

In [2]:
from paper.config import *

by_dir

'/Users/oyvsyo/proj/jooble/IR/jooble-ir-2018-paper/Data/by'

In [4]:
list_json = []

start_date = date(2017, 1, 1)
# end_date = date(2017, 1, 30)
end_date = date(2017, 1, 1)

date_range = pd.date_range(start_date, end_date)
    
#looping through all days from 2017_01_1 to 2017_01_30 with step = 1 day
for single_date in date_range:
    date_ = single_date.strftime("%Y-%m-%d")
        
    #!!!!CHANGE PATH!!!!!!
    file_job = os.path.join(by_dir, 'by_job_{}.json.gz'.format(date_))
    file_job_text = os.path.join(by_dir, 'by_job_text_{}.json.gz'.format(date_))

    #reading two files simuntaneously
    document_job, document_job_text = [],[]
    with gzip.open(file_job, "rb") as f1, gzip.open(file_job_text, "rb") as f2: 
        for line1, line2 in zip(f1, f2):
            vacation_job = json.loads(line1)
            vacation_job_text = json.loads(line2)
            document_job.append(vacation_job)
            document_job_text.append(vacation_job_text)

        #creating a list of dictionaries
        for i in range(len(document_job)):
            for j in range(len(document_job_text)):
                if document_job[i]['id'] == document_job_text[j]['id_job']:
                    data = {}
                    data['id'] = document_job[i]['id']
                    data['lang_text'] = document_job[i]['lang_text']
                    data['lang_title'] = document_job[i]['lang_title']
                    data['company_name'] = document_job[i]['company_name']
                    data['title'] = document_job[i]['title']
                    data['title_normalized'] = document_job[i]['title_normalized']
                    data['salary_val1'] = document_job[i]['salary_val1']
                    data['salary_val2'] = document_job[i]['salary_val2']
                    data['company_name'] = document_job[i]['company_name']
                    data['url'] = document_job[i]['url']
                    
                    data['text'] = document_job_text[j]['text']
                    list_json.append(data)
                    
print(len(list_json))

219


## Make Index update via `updater` service

In [6]:
from paper.utils import post_request as post


update_json = {'docs': list_json}

j = post(UPDATER_PORT, UPDATER_PATH, update_json)

## Develop redis tf-idf

In [7]:
stmdocs = j['hh']

In [50]:
# stemmed, docs = stmdocs[1][:10], stmdocs[0][:10]
stemmed, docs = stmdocs[1][10:30], stmdocs[0][10:30]
# stemmed, docs = stmdocs[1][30:100], stmdocs[0][30:100]

In [60]:
# stemmed, docs = [['test1','test2'],], [{"1":1}]
stemmed, docs = [['test2', 'test3'],], [{"2":2}]

In [61]:
from redis import Redis
import pandas as pd

redis_conn = Redis(decode_responses=True)
routine_pipe = redis_conn.pipeline()
pipe = redis_conn.pipeline()

In [62]:
from collections import Counter

test_tok = 'от'
# redis_conn.flushall()

N = 10e9
max_id = pipe.get("max doc id").execute()[0]
if max_id is None:
    max_id = 0
else:
    max_id = int(max_id)

new_ids = range(max_id, len(docs), 1)

vocabulary = set(sum(stemmed, []))

#  calculate document frequency for each token
#  local_df - {token: df}
local_df = {}
for stm_doc in stemmed:
    for tok in vocabulary:
        if tok in stm_doc:
            try:
                local_df[tok] += 1
            except:
                local_df[tok] = 1

#  local_tf_dict - {doc_id: {token: tf}}
#  tf for new documents
local_tf_dict = {}
for doc, stm_doc, doc_id in zip(docs, stemmed, new_ids):    
    local_tf_dict[doc_id] = Counter(stm_doc)               

#  inverted index for new documents
#  invindex - {tok: {doc_id, ...} }
texts = {i:stem for i, stem in zip(new_ids, stemmed)}
invindex = {word:set(txt for txt, wrds in texts.items() if word in wrds)
            for word in vocabulary}

                
#  existingt tf-idf for each token in vocabulary to change
#  { tok: [(doc_id, tfidf)], .. }
for tok in vocabulary:
    routine_pipe.zrange(tok, 0, -1, withscores=True, score_cast_func=float)
buff = routine_pipe.execute()
old_table_vocab = {tok:lst for tok,lst in zip(vocabulary,buff)}

# get old df from redis
# old_df_dict - {tok: df} (df - len of docs in redis for token tok)
for tok in vocabulary:
    routine_pipe.zcount(tok, '-inf', '+inf')
buff = routine_pipe.execute()
old_df_dict = {tok:df for tok, df in zip(vocabulary,buff)}

#  now we can iterate througth new vocabulary and docs to 
#  calculate new tf-idf
new_index = {}
for tok in vocabulary:
    new_index[tok] = {}
    
    loc_df = local_df[tok]
    old_df = old_df_dict[tok]
    new_df = loc_df + old_df   # count of all docs with token tok
    loc_post_list = invindex[tok]   # set of new doc ids where tok is
    old_tfidf_tuples = old_table_vocab[tok]   # list with pairs (old_doc_id, tf-idf)
    
    #  calculate tf_idf for new docs and vocab
    for new_doc_id in loc_post_list:
        tf = local_tf_dict[new_doc_id][tok]
        new_tfidf = (1 + log(tf, 10)) * log(N/loc_df, 10)
        print('-'*40, '\ntok = ', tok,'\ntf = ', tf,
              '\n0ld_df = ', old_df, '\ntfidf = ', new_tfidf,
              '\nloc_df = ', loc_df,
              '\nnew_doc_id =', new_doc_id)
        new_index[tok][new_doc_id] = new_tfidf
        
    #  recalculate tf_idf for existing docs in redis
    for old_doc_id, old_tf_idf in old_tfidf_tuples:
        new_tfidf = old_tf_idf * log(N/new_df, 10) / log(N/old_df, 10)
        print('+'*40,'\ntok = ', tok,
              '\n0ld_df = ', old_df, '\ntfidf = ', new_tfidf,
              '\nold_tfidf = ', old_tf_idf,
              '\nloc_df = ', loc_df,
              '\nold_doc_id =', old_doc_id)
        new_index[tok][old_doc_id] = new_tfidf
    
    [pipe.zadd(tok, d_id, new_index[tok][d_id]) for d_id in new_index[tok].keys()]
    pipe.execute()
    
for new_doc, new_id in zip(docs, new_ids):
    pipe.hmset("doc:{}".format(new_id), new_doc)
pipe.incrby("max doc id", len(docs))

---------------------------------------- 
tok =  test2 
tf =  1 
0ld_df =  1 
tfidf =  10.0 
loc_df =  1 
new_doc_id = 0
++++++++++++++++++++++++++++++++++++++++ 
tok =  test2 
0ld_df =  1 
tfidf =  9.698970004336017 
old_tfidf =  10.0 
loc_df =  1 
old_doc_id = 0
---------------------------------------- 
tok =  test3 
tf =  1 
0ld_df =  0 
tfidf =  10.0 
loc_df =  1 
new_doc_id = 0


Pipeline<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>

In [32]:
old_df_dict

{'внедря': 0,
 'примерн': 0,
 'обновлен': 0,
 'располож': 0,
 'час': 0,
 'газов': 0,
 'над': 0,
 'стабильн': 0,
 'сад': 0,
 'при': 0,
 'исполнен': 0,
 'жительств': 0,
 'виде': 0,
 'времен': 0,
 'результат': 0,
 'rabot': 0,
 'текст': 0,
 'центр': 0,
 'вицин': 0,
 'модернизм': 0,
 '200кв': 0,
 'PayPa': 0,
 'ид': 0,
 'прибор': 0,
 'кандидат': 0,
 'добав': 0,
 'майнинг': 0,
 'частичн': 0,
 'знающ': 0,
 'ремонт': 0,
 'минут': 0,
 'бирж': 0,
 'статус': 0,
 'написан': 0,
 'прода': 0,
 'упуст': 0,
 'взросл': 0,
 'рерайт': 0,
 'яблон': 0,
 'скайп': 0,
 'автоальянс': 0,
 'отдела': 0,
 'мета': 0,
 'стил': 0,
 'сервак': 0,
 'огород': 0,
 'предусмотр': 0,
 'оклад': 0,
 'ту': 0,
 'turrbomax': 0,
 'некотор': 0,
 'основн': 0,
 'горизонтальн': 0,
 'включа': 0,
 'мандарин': 0,
 '2000': 0,
 'контакт': 0,
 'пример': 0,
 'слов': 0,
 'analytics': 0,
 'дизайн': 0,
 'дан': 0,
 'занят': 0,
 'мус': 0,
 'мозыр': 0,
 'он': 0,
 'газ': 0,
 'растен': 0,
 'наш': 0,
 'связ': 0,
 '7352352': 0,
 '58': 0,
 'построен': 0,

In [16]:
for tok in vocabulary:
    routine_pipe.zrange(tok, 0, -1, withscores=True, score_cast_func=float)
tfidf_table_vocab = routine_pipe.execute()
tfidf_table_vocab

[[('38', 10.0)],
 [('13', 9.522878745280336)],
 [('14', 9.698970004336017)],
 [('36', 10.0)],
 [('9', 8.999999999999998)],
 [('36', 10.0)],
 [('36', 9.522878745280336)],
 [('37', 9.698970004336017)],
 [('57', 10.0)],
 [('3', 9.397940008672037)],
 [('63', 10.0)],
 [('15', 10.0)],
 [('24', 10.0)],
 [('62', 10.0)],
 [('6', 9.096910013008054)],
 [('9', 9.221848749616356)],
 [('2', 12.227001848732938)],
 [('9', 9.154901959985741)],
 [('38', 14.771212547196624)],
 [('38', 10.0)],
 [('13', 10.0)],
 [('20', 10.0)],
 [('36', 10.0)],
 [('3', 13.010299956639813)],
 [('37', 9.698970004336017)],
 [('62', 10.0)],
 [('48', 10.0)],
 [('9', 9.154901959985741)],
 [('20', 9.698970004336017)],
 [('6', 9.698970004336017)],
 [('44', 10.0)],
 [('48', 10.0)],
 [('48', 13.010299956639813)],
 [('2', 12.618650902686372)],
 [('2', 14.326554742293187)],
 [('15', 10.0)],
 [('11', 10.0)],
 [('64', 10.0)],
 [('49', 9.698970004336017)],
 [('5', 10.0)],
 [('2', 10.0)],
 [('36', 10.0)],
 [('13', 10.0)],
 [('13', 9.52287

In [334]:
tok

'Cit'

In [335]:
for tok in stm_counter.keys():
    routine_pipe.zcount(tok, '-inf', '+inf')
old_df = routine_pipe.execute()

In [336]:
old_df

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [278]:
len(redis_conn.keys())

1332

In [343]:
redis_conn.zrange('от', 0, -1, withscores=True, score_cast_func=float)

[('2', 0.40027726599947855)]

In [322]:
redis_conn.zrange('водител', 0, -1, withscores=False, score_cast_func=float)

['17']

In [344]:
redis_conn.hgetall('doc:2')["text"]

'Примеры статей avtoaliance.ru/stati/ \nОсновная суть обзора - описание преимуществ работы с компанией Автоальянс выкуп авто на примере конкретного выкупа конкретной машины. Статья пишется от лица представителя компании,который осуществил выездную оценку и выкуп авто. Повествует о там как компания помогла человеку быстро и выгодно продать автомобиль. Перед написанием я скидываю Вам фото машины для ознакомления.\nПриветствуется наличие какой-то изюминки и нестандартный подход к написанию (юмор сатира и прочее) Примеры удачных статей \n\n\n\nОколотематические слова, некоторые из которых желательно употребить в тексте\nавтовыкуп \nавтоскупка \nвыкуп авто \nвыкуп автомобилей \nсрочный выкуп авто \nвыкуп авто в Москве \nМосковской Области \nскупка автомобилей \nвыкуп + марка авто (например выкуп Хонда, выкуп Honda) \nпродать авто \nбыстро продать автомобиль \nи т.д. и т.п. \nКаждый текст - 1000-1300 символов пишите стоимость за 1000 символов'

In [348]:
redis_conn.zcount('от', '-inf', '+inf')

1

In [347]:
stm_counter.keys()

dict_keys(['крупн', 'образовательн', 'центр', '"', 'лидер', 'приглаша', 'к', 'сотрудничеств', 'преподавател', 'курс', 'секретар', '-', 'референт', 'в', 'молодечн', 'требован', ':', 'коммуникабельн', 'стремлен', 'дел', 'знан', 'и', 'работа', 'на', 'результат', 'занят', 'частичн', 'по', '2', '3', 'час', 'раз', 'недел', 'мы', 'предлага', 'официальн', 'заработн', 'плат', 'комфортн', 'услов', 'труд', 'присыла', 'резюм', 'адрес', 'rabot', '@', 'lider', 'by', 'сайт'])

In [88]:
for tok in stm_counter.keys():
    pipe.zrange(tok, 0, -1, withscores=True, score_cast_func=float)
    print(tok)
old_tfidf = pipe.execute()


срочн
отрисова
не
сложн
ч
б
логотип
в
вектор


In [172]:
old_tfidf

[[], [], [], [], [], [], [], [('1', 0.0)], []]

In [71]:
for i, tok in enumerate(stm_counter.keys()):
    print(i, tok)

0 вопрос
1 задан
2 "
3 состав
4 протокол
5 наблюден
6 за
7 гендерн
8 специфик
9 руководител
10 обь
11 3
12 -
13 5
14 страниц


In [85]:
redis_conn.zcount('jj', '-inf', '+inf')

0

In [79]:
vocabulary = set(sum(stemmed, []))

new_df = {}
for stm_doc in stemmed:
    for tok in vocabulary:
        if tok in stm_doc:
            try:
                new_df[tok] +=1
            except:
                new_df[tok] = 1

In [195]:
for tok in stm_counter.keys():
    routine_pipe.zrange(tok, 0, -1, withscores=True, score_cast_func=float)
old_tfidf_table = routine_pipe.execute()

In [242]:
from collections import Counter

test_tok = 'от'
redis_conn.flushall()

N = 10e9
max_id = pipe.get("max doc id").execute()[0]
if max_id is None:
    max_id = 0
else:
    max_id = int(max_id)

new_ids = range(max_id, len(docs), 1)

vocabulary = set(sum(stemmed, []))

#  calculate document frequency for each token
new_df = {}
for stm_doc in stemmed:
    for tok in vocabulary:
        if tok in stm_doc:
            try:
                new_df[tok] += 1
            except:
                new_df[tok] = 1

for doc, stm_doc, doc_id in zip(docs, stemmed, new_ids):
        
    stm_counter = Counter(stm_doc)
    #  get tf-idf of existing docs
    for tok in stm_counter.keys():
        routine_pipe.zrange(tok, 0, -1, withscores=True, score_cast_func=float)
    old_tfidf_table = routine_pipe.execute()
    
    #  get df for all tokens in document (length of redis token set)
    for tok in stm_counter.keys():
        routine_pipe.zcount(tok, '-inf', '+inf')
    old_df = routine_pipe.execute()
        
    #  recalculate tf-idf for existing docs
    for i, tok in enumerate(stm_counter.keys()):
        if tok==test_tok:
            print(old_df[i])
            print('i = ', i)
            print(stm_counter)
        if old_df[i] == 0:
            d_id = doc_id
            tf = stm_counter[tok]
            new_tfidf = (1 + log(tf, 10)) * log(N/new_df[tok], 10)
            if tok==test_tok:
                print('first time of tok ', tok)
                print('tf', tf)
                print('new_tfidf', new_tfidf)
            pipe.zadd(tok, d_id, new_tfidf)
        else:
            for doc_score in old_tfidf_table[i]:
                old_tfidf = doc_score[1]
                d_id = doc_score[0]
                new_df_tok = new_df[tok]+old_df[i]
                new_tfidf = old_tfidf * log(N/new_df_tok, 10) / log(N/old_df[i], 10)
                #  icrement count of docs with that particular tok
                if tok==test_tok:
                    print(tok)
                    print('id', d_id)
                    print('old doc frq', old_df[i])
                    print('new_df_tok', new_df_tok)
                    print('old tfidf', old_tfidf)
                    print('new tfidf', new_tfidf)
                    print('-'*20)
                old_df[i] = new_df_tok
                pipe.zadd(tok, d_id, new_tfidf)
    pipe.execute()
    #  update index
    pipe.hmset("doc:{}".format(doc_id), doc)
    
pipe.incrby("max doc id", len(docs))
pipe.execute();

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]